<a href="https://colab.research.google.com/github/smitdesai1010/BibliographyFactory-application/blob/master/COVID_19_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import json
import os
import shutil
import zipfile

!echo > /content/kaggle.json

token = {"username":"smitdesai1010","key":"41f928f2f4a0f0b462ed2ec45f383c82"}
with open('/content/kaggle.json', 'w') as file:
    json.dump(token, file)

!cp /content/kaggle.json ~/.kaggle/kaggle.json

!kaggle config set -n path -v /content
!chmod 600 /root/.kaggle/kaggle.json

!kaggle datasets download -d amanullahasraf/covid19-pneumonia-normal-chest-xray-pa-dataset

zf = zipfile.ZipFile('/content/datasets/amanullahasraf/covid19-pneumonia-normal-chest-xray-pa-dataset/covid19-pneumonia-normal-chest-xray-pa-dataset.zip') 
zf.extractall()

!rm -rf 'datasets' #Deleting the the downloaded Zip files after extracting it 


def create_dataset(name):
  org_path = '/content/'+ name
  train_path = '/content/train/'+ name
  val_path = '/content/validation/'+ name
  test_path = '/content/test/'+ name

  if not os.path.exists(train_path):
    os.mkdir(train_path)
    os.mkdir(val_path)
    os.mkdir(test_path)

  path, dirs, files = next(os.walk(org_path))
  file_count = len(files)
  i = 0  
    
  for file in os.listdir(org_path):
    
    if not os.path.isfile(os.path.join(org_path,file)):
      continue

    if i < file_count*0.75:
     shutil.copy2(os.path.join(org_path,file), os.path.join(train_path,file))
    
    elif i > file_count*0.9:
     shutil.copy2(os.path.join(org_path,file), os.path.join(test_path,file))

    else:
     shutil.copy2(os.path.join(org_path,file), os.path.join(val_path,file))
     
    i = i + 1 



if not os.path.exists('/content/train'):
  !mkdir /content/train
  !mkdir /content/validation
  !mkdir /content/test
 
create_dataset('covid')
create_dataset('normal')
create_dataset('pneumonia')

!rm -rf 'covid'
!rm -rf 'normal'
!rm -rf 'pneumonia'

- path is now set to: /content
100% 1.90G/1.91G [00:37<00:00, 70.6MB/s]
100% 1.91G/1.91G [00:37<00:00, 54.6MB/s]


https://medium.com/@mrgarg.rajat/training-on-large-datasets-that-dont-fit-in-memory-in-keras-60a974785d71

In [3]:
import tensorflow as tf
import cv2

import matplotlib.pyplot as plt
import numpy as np

from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop

# specifically for cnn
from keras.layers import Dropout, Flatten,Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization

from keras.preprocessing.image import ImageDataGenerator

In [24]:
img_dim = 128
batch_size = 64

train_generator = ImageDataGenerator(
    rescale = 1. / 255 ,
    horizontal_flip = True
)

val_generator = ImageDataGenerator(
    rescale = 1. / 255 ,
    horizontal_flip = True
)

test_generator = ImageDataGenerator(
    rescale = 1. / 255 ,
    horizontal_flip = True
)

train_data = train_generator.flow_from_directory(
    '/content/train' ,
    target_size = (img_dim,img_dim) ,
    batch_size = batch_size ,
)

val_data = val_generator.flow_from_directory(
    '/content/validation' ,
    target_size = (img_dim,img_dim) ,
    batch_size = batch_size ,
)

test_data = test_generator.flow_from_directory(
    '/content/test' ,
    target_size = (img_dim,img_dim) ,
    batch_size = batch_size ,
)

Found 5172 images belonging to 3 classes.
Found 1039 images belonging to 3 classes.
Found 691 images belonging to 3 classes.


In [25]:
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_dim, img_dim)
else:
    input_shape = (img_dim, img_dim, 3)

model=tf.keras.models.Sequential([
                                  
tf.keras.layers.Conv2D(32,(3,3),activation='relu',input_shape=input_shape),
tf.keras.layers.MaxPool2D(pool_size=(2,2)),

tf.keras.layers.Conv2D(16,(3,3),activation='relu'),
tf.keras.layers.BatchNormalization(),
tf.keras.layers.MaxPool2D(pool_size=(3,3)),

tf.keras.layers.Flatten(),
tf.keras.layers.Dense(16,activation='relu'),
tf.keras.layers.Dense(3,activation='sigmoid')                                 
                                  
])

model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_24 (Conv2D)           (None, 126, 126, 32)      896       
_________________________________________________________________
max_pooling2d_24 (MaxPooling (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_25 (Conv2D)           (None, 61, 61, 16)        4624      
_________________________________________________________________
batch_normalization_12 (Batc (None, 61, 61, 16)        64        
_________________________________________________________________
max_pooling2d_25 (MaxPooling (None, 20, 20, 16)        0         
_________________________________________________________________
flatten_8 (Flatten)          (None, 6400)              0         
_________________________________________________________________
dense_21 (Dense)             (None, 16)               

In [27]:
model.fit(
    train_data ,
    steps_per_epoch = 5178/batch_size ,
    epochs = 15 ,
    validation_data=val_data ,
    validation_steps = 1032/batch_size ,
    verbose = 1 
)

Epoch 1/15
81/80 [==============================] - 79s 972ms/step - loss: 0.2264 - accuracy: 0.9230 - val_loss: 1.1202 - val_accuracy: 0.7247
Epoch 2/15
81/80 [==============================] - 79s 970ms/step - loss: 0.2012 - accuracy: 0.9329 - val_loss: 0.5721 - val_accuracy: 0.8219
Epoch 3/15
81/80 [==============================] - 78s 959ms/step - loss: 0.1986 - accuracy: 0.9310 - val_loss: 0.2941 - val_accuracy: 0.8999
Epoch 4/15
81/80 [==============================] - 77s 950ms/step - loss: 0.1687 - accuracy: 0.9437 - val_loss: 0.5757 - val_accuracy: 0.8268
Epoch 5/15
81/80 [==============================] - 77s 951ms/step - loss: 0.1634 - accuracy: 0.9457 - val_loss: 0.6843 - val_accuracy: 0.8162
Epoch 6/15
81/80 [==============================] - 77s 953ms/step - loss: 0.1501 - accuracy: 0.9470 - val_loss: 0.3344 - val_accuracy: 0.8999
Epoch 7/15
81/80 [==============================] - 78s 959ms/step - loss: 0.1307 - accuracy: 0.9549 - val_loss: 0.2927 - val_accuracy: 0.9153